# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: Luis Antonio Pelayo Sierra

**Professor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Structured Streaming with Files") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 01:28:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
!pwd

/opt/spark/work-dir


In [ ]:
!ls

artifacts  data  datacar_service  examples  img  jars  labs  lib  libs	opt


In [5]:
%run lib/luis_pelayo/logs.py

[logger] Writing logs to: /opt/spark/work-dir/spark/data/logs
[logger] Wrote 3 lines to log_1760059785.log
[logger] Wrote 4 lines to log_1760059787.log
[logger] Wrote 3 lines to log_1760059796.log
[logger] Wrote 3 lines to log_1760059799.log
[logger] Wrote 3 lines to log_1760059804.log
[logger] Wrote 4 lines to log_1760059807.log
[logger] Wrote 3 lines to log_1760059816.log
[logger] Wrote 2 lines to log_1760059819.log
[logger] Wrote 1 lines to log_1760059827.log
[logger] Wrote 4 lines to log_1760059832.log
[logger] Wrote 2 lines to log_1760059837.log
[logger] Wrote 2 lines to log_1760059841.log


In [6]:
from luis_pelayo.spark_utils import SparkUtils
from pyspark.sql.functions import split, col

schema = SparkUtils.generate_schema([("Timestamp", "timestamp"),
                                        ("Level", "string"),
                                        ("Message", "string"),
                                        ("Host", "string")])

logs = spark.readStream \
    .format("text") \
    .load("/opt/spark/work-dir/data/logs/")  

logs_parsed = logs.select(split(col("value"), " \\| ").alias("fields")).select(col("fields").getItem(0).alias("Timestamp").cast("timestamp"),
                                                                                col("fields").getItem(1).alias("Level"),
                                                                                col("fields").getItem(2).alias("Message"),
                                                                                col("fields").getItem(3).alias("Host"))

logs_filtered = logs_parsed.filter((col("Level").isin("ERROR", "CRITICAL", "WARNING")))

query = logs_filtered.writeStream \
                        .outputMode("append") \
                        .format("console") \
                        .option("truncate", False) \
                        .start()

query.awaitTermination(60)

False

In [ ]:
sc.stop()